### Файлы проекта:

  - pipe.py - скрипт обработки данных и паплайн для сервиса
  - test_model.py - скрипт по тестированию полученной модели
  - api.py - скрипт обработки get и post запросов
  - data - папка для получаемых данных от отработки скрипта pipe.py
  - test_0.json, test_1.json - файлы для тестирования модели, результаты 0 и 1
  - missing_city.json вспомогательный файл для заполнения geo_city
  - структура data:
  ~ df_geo_city_loc_with_na.csv - файл с неполными данными по геолокациям,  
  ~ df_geo_city_loc.csv - файл с полными данными по геолокациям, 
  ~ папка df_likelihood содержит генерируемые промежуточные файлы df_likelihood_*.csv с данными для дальнейшей обработки данных


### Режимы работы:

1. (при отсутствии файлов с данными) формирование вспомогательных файлов для построения модели
2. (при наличии файлов с данными) проверка наличия файлов, формирование модели
3. (при наличии модели) предсказание с помощью localhost web app

Краткое описание порядка работы скрипта pipe.py:
1. чтение данных из исходных файлов
2. удаление дубликатов в  df_ga_hits.hit_number
3. объединение фреймов (left_on='session_id', right_on='session_id', how='inner')
4. преобразование строковой целевой переменной в числовую 0/1
5. удаление столбцов, не участвующих в дальнейшей модели
6. генерация файла df_geo_city_loc_with_na.csv с помощью сервиса Nominatim (geo* в координаты)
7. генерация файла df_geo_city_loc.csv с помощью ранее полученного файла и сервиса yandex
8. генерация файлов df_likelihood_*.csv (вероятности достижения целевой переменной 1 для каждой категориальной переменной)
9. пайплайн:
   - преобразование типа данных, заполнение пропусков
   - транформация столбцов geo
   - трансформация столбца device_screen
   - транформация столбцов utm*, device*
   - обучение модели
14. получение метрики модели
15. запись полученной модели в файл data/hit_predict.pkl

Краткое описание порядка работы скрипта test_model.py:
1. скачивание полученной модели data/hit_predict.pkl
2. тестирование предсказаний на файлах data/test_json_0.json и data/test_json_1.json

Краткое описание работы скрипта main.py:
1. обработка Get запроса на локальном хосте 127.0.0.1:8000/status - состояние 
2. обработка Get запроса на локальном хосте 127.0.0.1:8000/version - версия модели 
3. обработка Post запроса на локальном хосте 127.0.0.1:8000/predict - сервис предсказания 
по отдельному событию в числовом формате 0|1.

### Результаты:

#### полученные результаты метрик (evaluation, 5 folds):

- для модели LogisticRegression(): ROC_AUC.mean=0.606 ROC_AUC.std =0.001
- для модели RandomForestClassifier(): ROC_AUC.mean=0.759 ROC_AUC.std =0.001
- для модели MLPClassifier(): ROC_AUC.mean=0.646 ROC_AUC.std =0.003

#### лучшая модель на всей выборке:
- на всей выборке RandomForestClassifier ROC AUC = 0.864
- проверка: RandomForestClassifier(), 5 folds: ROC_AUC.mean=0.759 ROC_AUC.std =0.001